# Smart Cultural Storyteller - AI Content Generation

This notebook generates images and videos for the Betal storytelling project using free AI models.

## Setup and Dependencies

In [3]:
# Install required packages
%pip install requests pillow opencv-python moviepy huggingface-hub transformers torch diffusers
%pip install gTTS  
%pip install gradio python-dotenv
!

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\itske\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\itske\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\itske\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import requests
import json
import os
import time
from PIL import Image, ImageEnhance, ImageFilter
import cv2
import numpy as np
from moviepy.editor import *
#import torch
#from diffusers import StableDiffusionPipeline
from gtts import gTTS
import io
from pathlib import Path

## Configuration

In [5]:
# API Keys
from dotenv import load_dotenv
import os

load_dotenv()  # take variables from .env
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")


# Output directories
OUTPUT_DIR = Path("generated_content")
IMAGES_DIR = OUTPUT_DIR / "images"
AUDIO_DIR = OUTPUT_DIR / "audio"
VIDEOS_DIR = OUTPUT_DIR / "videos"

# Create directories
for dir_path in [IMAGES_DIR, AUDIO_DIR, VIDEOS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

## Story Data

## Image Generation using Stable Diffusion

In [6]:
def check_model_status(model_name):
    """Check if the model is loaded and ready"""
    API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {"Authorization": f"Bearer {HUGGINGFACE_TOKEN}"}
    
    response = requests.get(API_URL, headers=headers)
    print(f"Model status: {response.status_code}")
    if response.status_code == 200:
        return True
    return False

def generate_image_hf_api(prompt, filename, max_retries=3):
    """Generate image using Hugging Face Inference API with better error handling"""
    
    # Try different models if one fails
    models = [
        "black-forest-labs/FLUX.1-schnell"
    ]
    
    for model in models:
        API_URL = f"https://api-inference.huggingface.co/models/{model}"
        headers = {"Authorization": f"Bearer {HUGGINGFACE_TOKEN}"}
        
        # Enhanced prompt for Indian cultural context
        enhanced_prompt = f"{prompt}, Indian cultural motifs, traditional art style, highly detailed, rich textures, vivid harmonious colors, intricate patterns, cinematic soft lighting, emotional storytelling, high quality"
        
        payload = {
            "inputs": enhanced_prompt,
            "parameters": {
                "guidance_scale": 7.5,
                "num_inference_steps": 50,
                "width": 512,
                "height": 512
            }
        }
        
        for attempt in range(max_retries):
            try:
                print(f"Attempting to generate: {prompt[:50]}... (Model: {model}, Attempt: {attempt+1})")
                
                response = requests.post(API_URL, headers=headers, json=payload, timeout=60)
                
                if response.status_code == 200:
                    # Check if response is actually an image
                    if response.headers.get('content-type', '').startswith('image/'):
                        image_path = IMAGES_DIR / f"{filename}.png"
                        with open(image_path, "wb") as f:
                            f.write(response.content)
                        print(f"✅ Generated image: {image_path}")
                        return image_path
                    else:
                        print(f"❌ Response is not an image. Content-Type: {response.headers.get('content-type')}")
                        try:
                            error_data = response.json()
                            print(f"Error response: {error_data}")
                        except:
                            print(f"Raw response: {response.text[:200]}...")
                
                elif response.status_code == 503:
                    try:
                        error_data = response.json()
                        if "loading" in str(error_data).lower():
                            estimated_time = error_data.get("estimated_time", 30)
                            print(f"⏳ Model is loading. Waiting {estimated_time} seconds...")
                            time.sleep(estimated_time + 10)  # Wait a bit longer
                            continue
                    except:
                        pass
                    print(f"❌ Service unavailable: {response.text}")
                
                else:
                    print(f"❌ Error {response.status_code}: {response.text}")
                
                # Wait before retry
                if attempt < max_retries - 1:
                    wait_time = (attempt + 1) * 5
                    print(f"⏳ Waiting {wait_time} seconds before retry...")
                    time.sleep(wait_time)
                    
            except requests.exceptions.Timeout:
                print(f"⏳ Request timeout. Retrying...")
                time.sleep(10)
            except Exception as e:
                print(f"❌ Exception: {str(e)}")
                time.sleep(5)
        
        print(f"❌ Failed with model {model}, trying next model...")
        time.sleep(5)
    
    print("❌ All models failed")
    return None

def test_api_connection():
    """Test the API connection and token validity"""
    headers = {"Authorization": f"Bearer {HUGGINGFACE_TOKEN}"}
    test_url = "https://api-inference.huggingface.co/models/runwayml/stable-diffusion-v1-5"
    
    try:
        response = requests.get(test_url, headers=headers)
        print(f"API Test - Status Code: {response.status_code}")
        if response.status_code == 401:
            print("❌ Invalid or missing token. Please check your HUGGINGFACE_TOKEN")
            return False
        elif response.status_code == 200:
            print("✅ API connection successful")
            return True
        else:
            print(f"⚠️ Unexpected response: {response.text}")
            return True  # Might still work
    except Exception as e:
        print(f"❌ API test failed: {str(e)}")
        return False

In [7]:
def generate_story_images(story_data):
    """Generate images for the story with better error handling"""
    
    # Test API connection first
    if not test_api_connection():
        print("❌ API connection test failed. Please check your token.")
        return []
    
    image_paths = []
    
    for i, scene_prompt in enumerate(story_data["scenes"]):
        print(f"\n🎨 Generating image {i+1}/{len(story_data['scenes'])}")
        filename = f"{story_data['id']}_scene_{i+1:02d}"
        image_path = generate_image_hf_api(scene_prompt, filename)
        
        if image_path:
            image_paths.append(str(image_path))
        else:
            print(f"❌ Failed to generate image for scene {i+1}")
        
        # Add delay to avoid rate limiting
        print("⏳ Waiting to avoid rate limiting...")
        time.sleep(10)  # Increased delay
    
    return image_paths


## Audio Generation (Text-to-Speech)

In [8]:

def generate_audio_gtts(text, filename, lang='en', tld='co.in'):
    """Generate audio using Google Text-to-Speech with Indian accent (male-sounding)"""
    tts = gTTS(text=text, lang=lang, tld=tld, slow=True)  # Slower speech for deeper tone
    audio_path = AUDIO_DIR / f"{filename}.mp3"
    tts.save(str(audio_path))
    print(f"Generated audio: {audio_path}")
    return audio_path



## Video Generation with Effects

In [9]:
from pathlib import Path
import os, re
import numpy as np
from moviepy.editor import (
    ImageClip,
    CompositeVideoClip,
    concatenate_videoclips,
    AudioFileClip,
)

def _safe_name(name: str) -> str:
    base = os.path.splitext(os.path.basename(str(name)))[0]
    return re.sub(r'[^A-Za-z0-9._-]+', '_', base)[:80]

def ease_in_out_cubic(x: float) -> float:
    return 4 * x * x * x if x < 0.5 else 1 - (-2 * x + 2) ** 3 / 2

import cv2
import numpy as np
import moviepy.editor as mp
def get_blurred_background_cv2(img_path, out_size, blur_ksize=51):
    img = cv2.imread(img_path)
    img = cv2.resize(img, out_size)
    img = cv2.GaussianBlur(img, (blur_ksize, blur_ksize), 0)
    tmp_path = "temp_blur.jpg"
    cv2.imwrite(tmp_path, img)
    return mp.ImageClip(tmp_path).set_duration(5)


def add_parallax_effect(image_path, duration=5, out_size=(1280, 720), variant_idx=0, zoom=0.04, pan_px=30):
    from moviepy.editor import ImageClip, CompositeVideoClip, vfx
    import numpy as np

    def ease_in_out_cubic(x):
        return 4*x*x*x if x < 0.5 else 1 - (-2*x + 2) ** 3 / 2

    base = ImageClip(image_path).set_duration(duration)
    W, H = out_size
    img_w, img_h = base.size

    # scale image to fit inside frame (letterbox style)
    fit_scale = min(W / img_w, H / img_h)
    scale_start = fit_scale
    scale_end   = fit_scale * (1.0 + max(0.0, zoom))

    def scale_fn(t):
        p = ease_in_out_cubic(t / duration)
        return scale_start + (scale_end - scale_start) * p

    rng = np.random.RandomState(variant_idx or 0)
    pan_x = int(rng.choice([-1, 1]) * rng.randint(max(12, pan_px // 2), pan_px))
    pan_y = int(rng.choice([-1, 1]) * rng.randint(max(8, pan_px // 3), max(14, pan_px)))

    def pos_fn(t):
        p = ease_in_out_cubic(t / duration)
        # center alignment + parallax pan
        return (
            (W - img_w * scale_fn(t)) / 2 - pan_x * (1 - p),
            (H - img_h * scale_fn(t)) / 2 - pan_y * (1 - p),
        )

    # Foreground (main image, centered and animated)
    moving = base.resize(lambda t: scale_fn(t))

    # Background (blurred, scaled to cover full frame)
    background = get_blurred_background_cv2(image_path, out_size)


    return CompositeVideoClip(
        [background, moving.set_position(pos_fn)],
        size=out_size
    ).set_duration(duration)


def create_story_video(
    image_paths,
    audio_path,
    story_id,
    out_size=(1280, 720),
    fps=24,
    crossfade=0.3,       # set 0.0 to disable zoom entirely        # set 0 to disable pan entirely
    videos_dir=None,
):
    # Output directory
    if videos_dir is None:
        try:
            output_dir = Path(VIDEOS_DIR)
        except NameError:
            output_dir = Path("generated_content") / "videos"
    else:
        output_dir = Path(videos_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Audio
    if not (audio_path and os.path.exists(str(audio_path))):
        print("Audio file missing; provide valid audio_path.")
        return None
    audio_clip = AudioFileClip(str(audio_path))
    total_duration = audio_clip.duration

    n = len(image_paths)
    if n == 0:
        print("No images provided.")
        return None

    per = total_duration / n

    clips = []
    for i, img_path in enumerate(image_paths):
        if not os.path.exists(img_path):
            print(f"Image not found: {img_path}")
            continue

        start = i * per
        dur = per if i < n - 1 else max(0.1, total_duration - start)

        clip = add_parallax_effect(
            img_path,
            duration=dur,
            out_size=out_size,
            variant_idx=i
        )
        clip = clip.set_start(start)
        if i > 0 and crossfade > 0:
            clip = clip.crossfadein(min(crossfade, dur / 2))
        clips.append(clip)

    if not clips:
        print("No valid clips to compose.")
        return None

    timeline = concatenate_videoclips(clips, method="compose", padding=0)
    timeline = timeline.set_audio(audio_clip).set_fps(fps)

    safe_id = _safe_name(story_id)
    video_path = output_dir / f"{safe_id}_video.mp4"
    timeline.write_videofile(
        str(video_path),
        codec="libx264",
        audio=True,
        audio_codec="aac",
        fps=fps,
        remove_temp=True,
    )
    return str(video_path)

## Generate Subtitles

In [10]:
def generate_subtitles(story_content, audio_duration):
    """Generate subtitle timing for the story"""
    sentences = story_content.replace('\n', ' ').split('. ')
    sentences = [s.strip() + '.' for s in sentences if s.strip()]
    
    duration_per_sentence = audio_duration / len(sentences)
    
    subtitles = []
    current_time = 0
    
    for sentence in sentences:
        subtitle = {
            "start": round(current_time, 1),
            "end": round(current_time + duration_per_sentence, 1),
            "text": sentence
        }
        subtitles.append(subtitle)
        current_time += duration_per_sentence
    
    return subtitles



## Export Data for Web Application

## Batch Process All Stories

In [11]:
# Function to process all 10 stories
def process_all_stories():
    """Process all stories in the dataset"""
    
    # You can define all 10 stories here or load from external file
    stories = [
  {
    "id": "wisdom-1",
    "title": "The Monkey and the Crocodile",
    "theme": "wisdom",
    "content": "A clever monkey lived on a jamun tree by a river. One day, a crocodile tried to trick the monkey into coming onto his back to eat him. Using his wit, the monkey told the crocodile he had left his heart on the tree and must return to fetch it. The crocodile, fooled, brought him back to safety. The monkey's intelligence saved his life, proving that sharp thinking can triumph over brute strength.",
    "scenes": [
      "Riverbank view: the crocodile lurking beneath shimmering water, sun glinting on scales, monkey perched nervously on a jamun branch above",
      "Close encounter: monkey conversing with crocodile, subtle smirk, tension in the posture, shaded river reflections",
      "The clever trick: monkey pointing at a distant tree, expression animated, crocodile turning to look, sunlight catching ripples",
      "Escape sequence: monkey leaping from crocodile's back to branch, dramatic splash, dynamic motion capture",
      "Safe on tree: monkey sitting amidst jamun fruits, triumphant, warm golden sunset, calm river background"
    ]
  },
  {
    "id": "courage-1",
    "title": "The Brave Potter's Son",
    "theme": "courage",
    "content": "In a small village, a young potter's son discovered a fire threatening the granary at night. Everyone was asleep, but he ran through the smoke and carried buckets of water tirelessly until the fire died out. His courage saved the village's food supply and taught everyone that even the smallest hands can achieve great deeds when fear is ignored.",
    "scenes": [
      "Night village: moonlit huts with smoke rising from granary, quiet tension, flickering shadows",
      "The son grabs buckets: close-up on determined eyes, hands gripping clay buckets, dynamic firelight reflections",
      "Carrying water: motion blur as he runs, sparks flying, villagers half-awake in background",
      "Fire dying out: water splashing, flames retreating, dramatic light contrast",
      "Village sunrise: villagers cheering the exhausted boy, relief and pride on faces, soft warm morning glow"
    ]
  },
  {
    "id": "kindness-1",
    "title": "The Foolish Sage and the Hungry Crow",
    "theme": "kindness",
    "content": "A wise but absent-minded sage often forgot to eat. One day, he found a hungry crow pecking at grains near his hut. Instead of scolding it, he shared his meager food. The crow, grateful, later returned with a treasure hidden in the forest, showing that even small acts of kindness can bring unexpected blessings.",
    "scenes": [
      "Sage's hut: humble interior, soft morning light, sage arranging a few grains in a clay bowl",
      "Crow pecking: close-up of black feathers glinting, tiny grains scattered around, curious eyes",
      "Sharing food: sage extending bowl to crow, warm tones, gentle expressions",
      "Crow returning: dramatic forest backdrop, crow dropping a sparkling treasure near sage",
      "Resolution: sage amazed, villagers peeking curiously, sunlight highlighting newfound fortune"
    ]
  },
  {
    "id": "justice-1",
    "title": "The Clever Washerman",
    "theme": "justice",
    "content": "A dishonest rich man claimed that a poor washerman had stolen his gold coins. The village elders were about to punish the washerman, but he suggested testing the claim: they should weigh the washerman and the coins together, then separately. By clever reasoning, he revealed the man had hidden the coins in his own pockets. The washerman’s calm intelligence exposed injustice and taught the value of fairness and wit.",
    "scenes": [
      "Village square: elders and villagers gathered, tense expressions, sun casting long shadows",
      "Rich man accusing: proud stance, coins glittering, dramatic light highlighting greed",
      "Washerman's suggestion: calm and composed, pointing at scale, everyone listening, focus on reasoning",
      "Revealing deception: coins falling from rich man's pockets, shocked faces, dynamic movement",
      "Resolution: villagers cheering, washerman respected, bright balanced daylight emphasizing fairness"
    ]
  }
]



    
    processed_stories = []
    
    for story in stories:
        print(f"\n=== Processing Story: {story['title']} ===")
        
        try:
            # Generate images
            image_paths = generate_story_images(story)
            
            # Generate audio
            narration_text = story["content"].replace('\n', ' ').strip()
            audio_path = generate_audio_gtts(narration_text, f"{story['id']}_narration")
            
            # Generate video
            video_path = create_story_video(image_paths, audio_path, story['id'])
            
            # Generate subtitles
            audio_clip = AudioFileClip(str(audio_path))
            subtitles = generate_subtitles(story['content'], audio_clip.duration)
            
            # Create web data
            web_data = {
                "id": story['id'],
                "title": story['title'],
                "content": story['content'],
                "images": [f"/generated_content/images/{Path(p).name}" for p in image_paths],
                "audioUrl": f"/generated_content/audio/{audio_path.name}",
                "videoUrl": f"/generated_content/videos/{video_path}" if video_path else None,
                "subtitles": subtitles
            }
            
            processed_stories.append(web_data)
            print(f"✅ Successfully processed: {story['title']}")
            
        except Exception as e:
            print(f"❌ Error processing {story['title']}: {str(e)}")
    
    # Save all processed stories
    all_stories_file = OUTPUT_DIR / "all_stories_data.json"
    with open(all_stories_file, 'w') as f:
        json.dump(processed_stories, f, indent=2)
    
    print(f"\n🎉 Batch processing complete! Data saved to: {all_stories_file}")
    return processed_stories

# Uncomment to process all stories
processed_stories = process_all_stories()


=== Processing Story: The Monkey and the Crocodile ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Riverbank view: the crocodile lurking beneath shim... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\wisdom-1_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Close encounter: monkey conversing with crocodile,... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\wisdom-1_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: The clever trick: monkey pointing at a distant tre... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\wisdom-1_scene_03.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 4/5
Attempting to generate: Escape sequence: monkey leaping from crocodile's b... (Model

MoviePy - Done.
Moviepy - Writing video generated_content\videos\wisdom-1_video.mp4



Moviepy - Done !
Moviepy - video ready generated_content\videos\wisdom-1_video.mp4
✅ Successfully processed: The Monkey and the Crocodile

=== Processing Story: The Brave Potter's Son ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Night village: moonlit huts with smoke rising from... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\courage-1_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: The son grabs buckets: close-up on determined eyes... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\courage-1_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Carrying water: motion blur as he runs, sparks fly... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\courage-1_scene_03.png
⏳ Waiting

MoviePy - Done.
Moviepy - Writing video generated_content\videos\courage-1_video.mp4



Moviepy - Done !
Moviepy - video ready generated_content\videos\courage-1_video.mp4
✅ Successfully processed: The Brave Potter's Son

=== Processing Story: The Foolish Sage and the Hungry Crow ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Sage's hut: humble interior, soft morning light, s... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\kindness-1_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Crow pecking: close-up of black feathers glinting,... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\kindness-1_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Sharing food: sage extending bowl to crow, warm to... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\kindness-1_scene_03.p

MoviePy - Done.
Moviepy - Writing video generated_content\videos\kindness-1_video.mp4



Moviepy - Done !
Moviepy - video ready generated_content\videos\kindness-1_video.mp4
✅ Successfully processed: The Foolish Sage and the Hungry Crow

=== Processing Story: The Clever Washerman ===
API Test - Status Code: 404
⚠️ Unexpected response: Not Found

🎨 Generating image 1/5
Attempting to generate: Village square: elders and villagers gathered, ten... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\justice-1_scene_01.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 2/5
Attempting to generate: Rich man accusing: proud stance, coins glittering,... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\justice-1_scene_02.png
⏳ Waiting to avoid rate limiting...

🎨 Generating image 3/5
Attempting to generate: Washerman's suggestion: calm and composed, pointin... (Model: black-forest-labs/FLUX.1-schnell, Attempt: 1)
✅ Generated image: generated_content\images\justice-1_scene_03.png
⏳

MoviePy - Done.
Moviepy - Writing video generated_content\videos\justice-1_video.mp4



Moviepy - Done !
Moviepy - video ready generated_content\videos\justice-1_video.mp4
✅ Successfully processed: The Clever Washerman

🎉 Batch processing complete! Data saved to: generated_content\all_stories_data.json


## Instructions for Integration

1. **Copy generated files** to your web project's `public/generated_content/` folder
2. **Update story data** in your React app with the generated JSON
3. **Test the video playback** in your web application
4. **Adjust timing and effects** as needed for better user experience

### File Structure for Web App:
```
public/
├── generated_content/
│   ├── images/
│   │   ├── wisdom-1_scene_1.png
│   │   ├── wisdom-1_scene_2.png
│   │   └── ...
│   ├── audio/
│   │   └── wisdom-1_narration.mp3
│   └── videos/
│       └── wisdom-1_video.mp4
```